This notebook serves as showcase for working with data stored in a (SQLite) Database and is based upon a german study assignment. The database used as well as a .CSV-file can be downloaded [here](https://www.dropbox.com/s/iislkxvmpklqqj5/sqlassignment_data.zip?dl=1).

# Assignments

1. Establish a connection to the local database ````laender.db````. Create lists for the table names and the included columns as well as the according data types.
2. Open the file ````armutsgefahr.txt```` in Python. Which data scheme should most likely be presented here?
3. Create a new table ```armutsgefahr``` in ```laender.db``` and three columns ```land, jahr, quote```. Fill these tables with the according data from ```armutsgefahr.txt```.
4. Enter a SQL Query for all government heads for whose government the SPD party is involved. Further restrict the query for the Old Federal States.
5. Enter a SQL Query containing the following elements.
    * voting weight for each country
    * absolute difference between residents in ````laender_info```` and residents in ```migrationshintergrund_2013```
    * percentage of people with a migration background in relation to the total population
    * restricted for countries with more than 200 residents per km$²$ but no more than 2000
6. Replace all missing values from column ```hauptstadt``` in the table ```laender_info```.

## Assignment 1

To work with the local SQLite database first a connection object as well as a cursor object are created.

In [1]:
%cd "E:\Dropbox\Soziologie Master\Medium Data\Aufgaben\Aufgabe 1"
import sqlite3

conn = sqlite3.connect('laender.db')
c = conn.cursor()

E:\Dropbox\Soziologie Master\Medium Data\Aufgaben\Aufgabe 1


Using the master table you can extract information like tables names and store them in a python list.

In [2]:
tables = c.execute('''SELECT name FROM sqlite_master\
                      WHERE type='table' ''').fetchall()

tables = [table[0] for table in tables]
print tables

[u'laender_info', u'migrationshintergrund_2013']


Now columns names and according data types are saved via list comprehensions.

In [3]:
countries = c.execute('''PRAGMA table_info(laender_info)''').fetchall()
countries[:2]

[(0, u'index', u'INTEGER', 0, None, 0), (1, u'land', u'TEXT', 0, None, 0)]

In [4]:
country_cols = [(col[1], col[2]) for col in countries]
country_cols

[(u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'kuerzel', u'TEXT'),
 (u'hauptstadt', u'TEXT'),
 (u'beitrittsdatum', u'INTEGER'),
 (u'regierungschef', u'TEXT'),
 (u'regierungspartei', u'TEXT'),
 (u'stimmgewicht', u'INTEGER'),
 (u'flaeche', u'INTEGER'),
 (u'einwohner_mio', u'REAL'),
 (u'einwohner_km2', u'INTEGER'),
 (u'auslaender', u'REAL'),
 (u'sprachen', u'TEXT')]

In [5]:
migration = c.execute('''PRAGMA table_info(migrationshintergrund_2013)''').fetchall()
migration[:2]

[(0, u'index', u'INTEGER', 0, None, 0), (1, u'land', u'TEXT', 0, None, 0)]

In [6]:
mig_cols = [(col[1], col[2]) for col in migration]
mig_cols

[(u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'insgesamt', u'INTEGER'),
 (u'ohne_mh', u'INTEGER'),
 (u'mit_mh', u'INTEGER')]

In the next step all lists with column information will be combined in one single list.

In [9]:
table_cols = country_cols + mig_cols
table_cols

[(u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'kuerzel', u'TEXT'),
 (u'hauptstadt', u'TEXT'),
 (u'beitrittsdatum', u'INTEGER'),
 (u'regierungschef', u'TEXT'),
 (u'regierungspartei', u'TEXT'),
 (u'stimmgewicht', u'INTEGER'),
 (u'flaeche', u'INTEGER'),
 (u'einwohner_mio', u'REAL'),
 (u'einwohner_km2', u'INTEGER'),
 (u'auslaender', u'REAL'),
 (u'sprachen', u'TEXT'),
 (u'index', u'INTEGER'),
 (u'land', u'TEXT'),
 (u'insgesamt', u'INTEGER'),
 (u'ohne_mh', u'INTEGER'),
 (u'mit_mh', u'INTEGER')]

## Assignment 2

To inspect the data structure of ```armutsgefahr.txt``` the fill will be opened in reading mode and the first part will be displayed as plain string output.

In [23]:
with open('armutsgefahr.txt', 'r') as f:
    finput = f.read()
    
print finput[:400]

Bundesland	2005	2006	2007	2008	2009	2010	2011	2012	2013
Deutschland	14.7	14.0	14.3	14.4	14.6	14.5	15.0	15.0	15.5
Altes Bundesgebiet ohne Berlin	13.2	12.7	12.9	13.1	13.3	13.3	13.8	13.9	14.4
Neue Bundeslaender mit Berlin	20.4	19.2	19.5	19.5	19.5	19.0	19.4	19.6	19.8
Baden-Wuerttemberg	10.6	10.1	10.0	10.2	10.9	11.0	11.1	11.1	11.4
Bayern	11.4	10.9	11.0	10.8	11.1	10.8	11.1	11.0	11.3
Berlin	19.7	17.0	17.


It is easy to see that the file-format is most likely CSV (comma-separated-value). Tabulators are delimiters while each line represents one observation unit.

## Assignment 3

For the third assignment a new tables is created within the database, containing columns to insert data from the CSV file.

In [5]:
c.execute('''CREATE TABLE armutsgefahr
             (pid INTEGER PRIMARY KEY,
             land TEXT,
              jahr INTEGER,
              quote REAL)''')

Now, the csv file will be read line by line and converted to a python list.

In [63]:
import csv
import itertools

data = []
with open('armutzsgefahr.txt', 'r') as f:
     reader = csv.reader(f, delimiter='\t')
     for row in reader:
        data.append(row)

The following block of code is basically munging the structure of the data to hand it over to the SQL database in the right form. Note however that there are much easier solutions for this task, e.g. in using the pandas package and its matrix features.

In [ ]:
laender = [row[0] for row in data[1:]]
quote = [value for row in data[1:]for value in row[1:]]
jahre = data[0][1:]
pid = range(1, len(laender)*len(jahre)+1)

tuples = list(itertools.product(laender,jahre))
land_dup = [val[0] for val in tuples]
jahr_dup = [val[1] for val in tuples]

combined =zip(pid, land_dup, jahr_dup, quote)

This is how the result looks like in Python:

In [102]:
print len(laender), len(jahre), len(laender) * len(jahre),  len(pid),
combined[:11]

19 9 171 171

[(1, 'Deutschland', '2005', '14.7'),
 (2, 'Deutschland', '2006', '14.0'),
 (3, 'Deutschland', '2007', '14.3'),
 (4, 'Deutschland', '2008', '14.4'),
 (5, 'Deutschland', '2009', '14.6'),
 (6, 'Deutschland', '2010', '14.5'),
 (7, 'Deutschland', '2011', '15.0'),
 (8, 'Deutschland', '2012', '15.0'),
 (9, 'Deutschland', '2013', '15.5'),
 (10, 'Altes Bundesgebiet ohne Berlin', '2005', '13.2'),
 (11, 'Altes Bundesgebiet ohne Berlin', '2006', '12.7')]

And finally we insert into the SQL table and recheck our results.

In [98]:
c.executemany('''INSERT INTO armutsgefahr VALUES (?,?,?,?)''', 
              combined)

In [101]:
c.execute('''SELECT * FROM armutsgefahr''').fetchall()[:11]

[(1, u'Deutschland', 2005, 14.7),
 (2, u'Deutschland', 2006, 14.0),
 (3, u'Deutschland', 2007, 14.3),
 (4, u'Deutschland', 2008, 14.4),
 (5, u'Deutschland', 2009, 14.6),
 (6, u'Deutschland', 2010, 14.5),
 (7, u'Deutschland', 2011, 15.0),
 (8, u'Deutschland', 2012, 15.0),
 (9, u'Deutschland', 2013, 15.5),
 (10, u'Altes Bundesgebiet ohne Berlin', 2005, 13.2),
 (11, u'Altes Bundesgebiet ohne Berlin', 2006, 12.7)]

## Assignment 4

For the correct query we need to combine information from the columns ```regierungschef, regierungspartei``` and ```land``` from the table ```laender_info```. Using the SQL command ```LIKE``` we can then search strings for contained substrings to select all government heads related to *SPD*.

In [110]:
c.execute('''SELECT regierungschef FROM laender_info
             WHERE regierungspartei LIKE '%SPD%' ''').fetchall()

[(u'Jens Boehrnsen (SPD)',),
 (u'Olaf Scholz (SPD)',),
 (u'Klaus Wowereit (SPD)',),
 (u'Stanislaw Tillich (CDU)',),
 (u'Reiner Haseloff (CDU)',),
 (u'Annegret Kramp-Karrenbauer (CDU)',),
 (u'Torsten Albig (SPD)',),
 (u'Bodo Ramelow (Die Linke)',),
 (u'Malu Dreyer (SPD)',),
 (u'Erwin Sellering (SPD)',),
 (u'Dietmar Woidke (SPD)',),
 (u'Hannelore Kraft (SPD)',),
 (u'Winfried Kretschmann (Gruene)',),
 (u'Stephan Weil (SPD)',)]

To further restrict the query for the Old Federal States there are several options. Here we will simply hand over a list to the SQL Query and exclude all data matching the list inputs.

In [153]:
ost = ['Brandenburg', 'Berlin', 'Thueringen', 'Sachsen',\
       'Sachsen-Anhalt', 'Mecklenburg-Vorpommern']

In [154]:
c.execute('''SELECT regierungschef FROM laender_info \
             WHERE regierungspartei LIKE '%SPD%'\
             AND land NOT IN (?, ?, ?, ?, ?, ?) ''', ost).fetchall()

[(u'Jens Boehrnsen (SPD)',),
 (u'Olaf Scholz (SPD)',),
 (u'Annegret Kramp-Karrenbauer (CDU)',),
 (u'Torsten Albig (SPD)',),
 (u'Malu Dreyer (SPD)',),
 (u'Hannelore Kraft (SPD)',),
 (u'Winfried Kretschmann (Gruene)',),
 (u'Stephan Weil (SPD)',)]

##  Assignment 5

For the larger SQL query information from several columns needs to be combined:

    stimmgewicht    
    insgesamt
    ohne_mh    
    mit_mh    
    einwohner_mio    
    einwohner_km2    

Additionally, the code below indicates various scalings: population is listed in billions in table ```laender_info``` and in thousands in ```migrationshintergrund_2013```. We have to consider this for the query.

In [104]:
c.execute('''
    SELECT laender_info.land, stimmgewicht, einwohner_mio, \
    insgesamt, ohne_mh, mit_mh, einwohner_km2 \
    FROM laender_info \
    NATURAL JOIN migrationshintergrund_2013 ''').fetchall()

[(u'Bremen', 3, 0.652, 655, 468, 187, 1576),
 (u'Hamburg', 3, 2.0, 1744, 1239, 504, 2366),
 (u'Berlin', 4, 3.0, 3396, 2493, 903, 3829),
 (u'Sachsen', 4, 4.0, 4044, 3854, 190, 225),
 (u'Sachsen-Anhalt', 4, 2.0, 2250, 2152, 97, 114),
 (u'Saarland', 3, 0.998, 992, 821, 172, 396),
 (u'Bayern', 6, 12.0, 12556, 9967, 2589, 178),
 (u'Schleswig-Holstein', 4, 3.0, 2810, 2458, 352, 179),
 (u'Thueringen', 4, 2.0, 2164, 2076, 88, 138),
 (u'Rheinland-Pfalz', 4, 4.0, 3991, 3168, 823, 202),
 (u'Hessen', 5, 6.0, 6029, 4351, 1678, 287),
 (u'Mecklenburg-Vorpommern', 3, 2.0, 1598, 1527, 71, 71),
 (u'Brandenburg', 4, 2.0, 2448, 2326, 123, 85),
 (u'Nordrhein-Westfalen', 6, 18.0, 17553, 13137, 4415, 523),
 (u'Baden-Wuerttemberg', 6, 11.0, 10599, 7644, 2955, 301),
 (u'Niedersachsen', 6, 8.0, 7784, 6393, 1391, 166)]

Combining all elements results in the following query, which also includes a ```CAST``` function to convert data types for proper percentages.

In [26]:
c.execute('''
    SELECT stimmgewicht, laender_info.land, ABS(einwohner_mio * 1000 - insgesamt),\
    CAST(mit_mh AS REAL)/insgesamt from laender_info \
    NATURAL JOIN migrationshintergrund_2013 \
    WHERE einwohner_km2 > 200 AND einwohner_km2 <= 2000 \ ''').fetchall()

[(3, u'Bremen', 3.0, 0.28549618320610687),
 (4, u'Sachsen', 44.0, 0.046983184965380814),
 (3, u'Saarland', 6.0, 0.17338709677419356),
 (4, u'Rheinland-Pfalz', 9.0, 0.20621398145828113),
 (5, u'Hessen', 29.0, 0.2783214463426771),
 (6, u'Nordrhein-Westfalen', 447.0, 0.25152395601891414),
 (6, u'Baden-Wuerttemberg', 401.0, 0.27879988678177187)]

The high differences in total population for two regions indicate different roundings for the number of residents in billions.

## Assignment 6

For the last assignment we inspect missing values by checking ```NULL``` values within the column ```hauptstadt```.

In [2]:
staedte = [stadt[0] for stadt in c.execute\
        ('''SELECT land FROM laender_info WHERE hauptstadt IS NULL ''')]

In [3]:
staedte

[u'Hamburg', u'Berlin']

The two misssing values appear for the city states *Berlin* and *Hamburg*. We can easily insert valid entries by using the information from the ```land``` column.

In [4]:
c.executescript('''
    UPDATE laender_info SET hauptstadt='Hamburg' \
    WHERE land =='Hamburg';
    UPDATE laender_info SET hauptstadt='Berlin' \
    WHERE land =='Berlin' ''')

To verify our results we again select the ```hauptstadt``` values. Everything worked as intended.

In [5]:
c.execute('''SELECT hauptstadt FROM laender_info\
             WHERE land IN (?,?) ''', staedte).fetchall()

[(u'Hamburg',), (u'Berlin',)]